In [ ]:
import os
os.environ['KMP_DUPLICATE_LIB_OK']='True'

import matplotlib.pyplot as plt
import numpy as np
import torch
import torch.nn.functional as F
import pandas as pd
from sklearn.preprocessing import StandardScaler
import math
from functools import partial
from torch import nn, einsum
from einops import rearrange, reduce
import time
import random

In [ ]:

seed_value = 2021   # 设定随机数种子

np.random.seed(seed_value)
random.seed(seed_value)
os.environ['PYTHONHASHSEED'] = str(seed_value)  # 为了禁止hash随机化，使得实验可复现。

torch.manual_seed(seed_value)     # 为CPU设置随机种子
torch.cuda.manual_seed(seed_value)      # 为当前GPU设置随机种子（只用一块GPU）
torch.cuda.manual_seed_all(seed_value)   # 为所有GPU设置随机种子（多块GPU）

torch.backends.cudnn.deterministic = True

In [ ]:
def exists(x):
    return x is not None

def default(val, d):
    if exists(val):
        return val
    return d() if callable(d) else d

def identity(t, *args, **kwargs):
    return t

In [ ]:
# small helper modules
#残差链接块
class Residual(nn.Module):
    def __init__(self, fn):
        super().__init__()
        self.fn = fn

    def forward(self, x, *args, **kwargs):
        return self.fn(x, *args, **kwargs) + x
#使用最近邻方法上采样
def Upsample(dim, dim_out = None):
    return nn.Sequential(
        nn.Upsample(scale_factor = 2, mode = 'nearest'),
        nn.Conv1d(dim, default(dim_out, dim), 3, padding = 1)
    )
#用卷积层下采样
def Downsample(dim, dim_out = None):
    return nn.Conv1d(dim, default(dim_out, dim), 4, 2, 1)


In [ ]:
#返回一个参数标准化的卷积层
class WeightStandardizedConv2d(nn.Conv1d):
    """
    https://arxiv.org/abs/1903.10520
    weight standardization purportedly works synergistically with group normalization
    """
    def forward(self, x):
        eps = 1e-5 if x.dtype == torch.float32 else 1e-3

        weight = self.weight
        mean = reduce(weight, 'o ... -> o 1 1', 'mean')
        var = reduce(weight, 'o ... -> o 1 1', partial(torch.var, unbiased = False))
        normalized_weight = (weight - mean) * (var + eps).rsqrt()

        return F.conv1d(x, normalized_weight, self.bias, self.stride, self.padding, self.dilation, self.groups)

#返回一个参数标准化的线性层
class LayerNorm(nn.Module):
    def __init__(self, dim):
        super().__init__()
        self.g = nn.Parameter(torch.ones(1, dim, 1))

    def forward(self, x):
        eps = 1e-5 if x.dtype == torch.float32 else 1e-3
        var = torch.var(x, dim = 1, unbiased = False, keepdim = True)
        mean = torch.mean(x, dim = 1, keepdim = True)
        return (x - mean) * (var + eps).rsqrt() * self.g

#返回一个参数标准化的线性层
class PreNorm(nn.Module):
    def __init__(self, dim, fn):
        super().__init__()
        self.fn = fn
        self.norm = LayerNorm(dim)

    def forward(self, x):
        x = self.norm(x)
        return self.fn(x)

# sinusoidal positional embeds
#返回时间序列的位置编码
class SinusoidalPosEmb(nn.Module):
    def __init__(self, dim):
        super().__init__()
        self.dim = dim

    def forward(self, x):
        device = x.device
        half_dim = self.dim // 2
        emb = math.log(10000) / (half_dim - 1)
        emb = torch.exp(torch.arange(half_dim, device=device) * -emb)
        emb = x[:, None] * emb[None, :]
        emb = torch.cat((emb.sin(), emb.cos()), dim=-1)
        return emb


In [ ]:
a=SinusoidalPosEmb(64)
b=torch.tensor([1,2,3,4,5,6,7,8,9,10])
a(b).shape

In [ ]:

# building block modules
#dim_out需要能够整除groups
class Block(nn.Module):
    def __init__(self, dim, dim_out, groups = 8):
        super().__init__()
        self.proj = nn.Conv1d(dim, dim_out, 3, padding = 1)
        self.norm = nn.GroupNorm(groups, dim_out)
        self.act = nn.SiLU()

    def forward(self, x, scale_shift = None):
        x = self.proj(x)
        x = self.norm(x)

        if exists(scale_shift):
            scale, shift = scale_shift
            x = x * (scale + 1) + shift

        x = self.act(x)
        return x


In [ ]:
a=Block(1,8)
x=torch.randn(64,1,50)
a(x).shape

In [ ]:

class ResnetBlock(nn.Module):
    def __init__(self, dim, dim_out, *, time_emb_dim = None, groups = 8):
        super().__init__()
        self.mlp = nn.Sequential(
            nn.SiLU(),
            nn.Linear(time_emb_dim, dim_out * 2)
        ) if exists(time_emb_dim) else None

        self.block1 = Block(dim, dim_out, groups = groups)
        self.block2 = Block(dim_out, dim_out, groups = groups)
        self.res_conv = nn.Conv1d(dim, dim_out, 1) if dim != dim_out else nn.Identity()

    def forward(self, x, time_emb = None):

        scale_shift = None
        if exists(self.mlp) and exists(time_emb):
            time_emb = self.mlp(time_emb)
            time_emb = rearrange(time_emb, 'b c -> b c 1')
            #.chunk在给定维度(轴)上将输入张量进行分块儿
            scale_shift = time_emb.chunk(2, dim = 1)

        h = self.block1(x, scale_shift = scale_shift)

        h = self.block2(h)

        return h + self.res_conv(x)

In [ ]:
class Attention(nn.Module):
    def __init__(self, dim, heads = 4, dim_head = 32):
        super().__init__()
        self.scale = dim_head ** -0.5
        self.heads = heads
        hidden_dim = dim_head * heads

        self.to_qkv = nn.Conv1d(dim, hidden_dim * 3, 1, bias = False)
        self.to_out = nn.Conv1d(hidden_dim, dim, 1)

    def forward(self, x):
        b, c, n = x.shape
        qkv = self.to_qkv(x).chunk(3, dim = 1)
        q, k, v = map(lambda t: rearrange(t, 'b (h c) n -> b h c n', h = self.heads), qkv)

        q = q * self.scale

        sim = einsum('b h d i, b h d j -> b h i j', q, k)
        attn = sim.softmax(dim = -1)
        out = einsum('b h i j, b h d j -> b h i d', attn, v)

        out = rearrange(out, 'b h n d -> b (h d) n')
        return self.to_out(out)


In [ ]:
# model
class Unet1D(nn.Module):
    def __init__(
        self,
        #表示模型中的隐藏维度大小
        dim,
        #表示输入经过初始卷积后的通道数
        init_dim = None,
        #表示模型输出的通道数
        out_dim = None,
        #表示每个下采样阶段的通道数倍增率
        dim_mults=(1, 2, 2),
        #表示输入数据的通道数
        channels = 3,
        #用来做group——norm
        resnet_block_groups = 8
    ):
        super().__init__()

        # determine dimensions
        input_channels = channels

        init_dim = default(init_dim, dim)
        self.init_conv = nn.Conv1d(input_channels, init_dim, 7, padding = 3)

        dims = [init_dim, *map(lambda m: dim * m, dim_mults)]
        in_out = list(zip(dims[:-1], dims[1:]))

        block_klass = partial(ResnetBlock, groups = resnet_block_groups)

        # time embeddings

        time_dim = dim * 4

        sinu_pos_emb = SinusoidalPosEmb(dim)
        fourier_dim = dim

        self.time_mlp = nn.Sequential(
            sinu_pos_emb,
            nn.Linear(fourier_dim, time_dim),
            nn.GELU(),
            nn.Linear(time_dim, time_dim)
        )

        # layers

        self.downs = nn.ModuleList([])
        self.ups = nn.ModuleList([])
        num_resolutions = len(in_out)

        for ind, (dim_in, dim_out) in enumerate(in_out):
            is_last = ind >= (num_resolutions - 1)

            self.downs.append(nn.ModuleList([
                block_klass(dim_in, dim_in, time_emb_dim = time_dim),
                block_klass(dim_in, dim_in, time_emb_dim = time_dim),
                Residual(PreNorm(dim_in, Attention(dim_in))),
                Downsample(dim_in, dim_out) if not is_last else nn.Conv1d(dim_in, dim_out, 3, padding = 1)
            ]))

        mid_dim = dims[-1]
        self.mid_block1 = block_klass(mid_dim, mid_dim, time_emb_dim = time_dim)
        self.mid_attn = Residual(PreNorm(mid_dim, Attention(mid_dim)))
        self.mid_block2 = block_klass(mid_dim, mid_dim, time_emb_dim = time_dim)

        for ind, (dim_in, dim_out) in enumerate(reversed(in_out)):
            is_last = ind == (len(in_out) - 1)

            self.ups.append(nn.ModuleList([
                block_klass(dim_out + dim_in, dim_out, time_emb_dim = time_dim),
                block_klass(dim_out + dim_in, dim_out, time_emb_dim = time_dim),
                Residual(PreNorm(dim_out, Attention(dim_out))),
                Upsample(dim_out, dim_in) if not is_last else  nn.Conv1d(dim_out, dim_in, 3, padding = 1)
            ]))

        default_out_dim = channels 
        self.out_dim = default(out_dim, default_out_dim)

        self.final_res_block = block_klass(dim * 2, dim, time_emb_dim = time_dim)
        self.final_conv = nn.Conv1d(dim, self.out_dim, 1)

    def forward(self, x, time):
        x = self.init_conv(x)
        r = x.clone()

        t = self.time_mlp(time)

        h = []

        for block1, block2, attn, downsample in self.downs:

            x = block1(x, t)
            h.append(x)

            x = block2(x, t)
            x = attn(x)
            h.append(x)

            x = downsample(x)

        x = self.mid_block1(x, t)
        x = self.mid_attn(x)
        x = self.mid_block2(x, t)
        
        for block1, block2, attn, upsample in self.ups:
            x = torch.cat((x, h.pop()), dim = 1)
            x = block1(x, t)

            x = torch.cat((x, h.pop()), dim = 1)
            x = block2(x, t)
            x = attn(x)

            x = upsample(x)

        x = torch.cat((x, r), dim = 1)

        x = self.final_res_block(x, t)
        return self.final_conv(x)

In [ ]:
#判别器
class Discriminator(nn.Module):
    def __init__(
        self,
        #表示模型中的隐藏维度大小
        dim,
        #表示输入经过初始卷积后的通道数
        init_dim = None,
        #表示每个下采样阶段的通道数倍增率
        dim_mults=(1,2,2),
        #表示输入数据的通道数
        channels = 3
    ):
        super().__init__()

        # determine dimensions
        input_channels = channels

        init_dim = default(init_dim, dim)
        self.init_conv = nn.Conv1d(input_channels, init_dim, 1)

        dims = [init_dim, *map(lambda m: dim * m, dim_mults)]
        in_out = list(zip(dims[:-1], dims[1:]))

        # time embeddings
        time_dim = dim * 4

        sinu_pos_emb = SinusoidalPosEmb(dim)
        fourier_dim = dim

        self.time_mlp = nn.Sequential(
            sinu_pos_emb,
            nn.Linear(fourier_dim, time_dim),
            nn.LeakyReLU(0.2),
            nn.Linear(time_dim, time_dim)
        )

        # layers

        self.downs = nn.ModuleList([])
        num_resolutions = len(in_out)

        for ind, (dim_in, dim_out) in enumerate(in_out):
            is_last = ind >= (num_resolutions - 1)

            self.downs.append(nn.ModuleList([
                ResnetBlock(dim_in, dim_in, time_emb_dim = time_dim),
                Downsample(dim_in, dim_out) if not is_last else nn.Conv1d(dim_in, dim_out, 3, padding = 1)
            ]))

        mid_dim = dims[-1]
        self.mid_block1 = ResnetBlock(mid_dim, mid_dim, time_emb_dim = time_dim)
        self.fc = nn.Sequential(
            nn.Linear(mid_dim*14, 1),
        )

    def forward(self, x, time):
        x = self.init_conv(x)
        t = self.time_mlp(time)
        for block1,downsample in self.downs:
            x = block1(x, t)
            x = downsample(x)

        x = self.mid_block1(x, t)
        x = x.view(x.shape[0],-1)
        x=self.fc(x)
        return torch.sigmoid(x)

In [ ]:
model=Discriminator(dim=32,channels=1)
print(model)

In [ ]:
x=torch.randn(128,1,56)
model=Discriminator(dim=32,channels=1)
x=model(x,torch.randn(128))
x.shape

In [ ]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

#设置一些超参数t，α，β
num_steps = 100
#制定每一步的beta，beta按照时间从小到大变化
# scale = 1000 / num_steps
# beta_start = scale * 0.0001
# beta_end = scale * 0.02
#betas = torch.linspace(beta_start,beta_end,num_steps)  # 使用linespace指定bata的值，也可以用其他方法
betas = torch.linspace(-6, 6, num_steps)
betas = torch.sigmoid(betas) * (0.5e-2 - 1e-5) + 1e-5

alphas = 1-betas
#累计乘法
alphas_prod = torch.cumprod(alphas,0)
alphas_bar_sqrt = torch.sqrt(alphas_prod)
one_minus_alphas_bar_log = torch.log(1 - alphas_prod)
one_minus_alphas_bar_sqrt = torch.sqrt(1 - alphas_prod)

#为了计算t-1
#alphas_bar_sqrt最后拼接一个1
alphas_bar_sqrt = torch.cat([alphas_bar_sqrt,torch.ones(1)])
#one_minus_alphas_bar_sqrt最后拼接一个0
one_minus_alphas_bar_sqrt = torch.cat([one_minus_alphas_bar_sqrt,torch.zeros(1)])

In [ ]:
# p_sample——根据预测出来的噪声的均值以及方差计算当前时刻的数据分布
def p_sample(model, x, t, betas, one_minus_alphas_bar_sqrt,device):
    """从x[T]采样t时刻的重构值"""
    model=model.to(device)
    t = torch.tensor([t]).to(device)

    coeff = (betas[t] / one_minus_alphas_bar_sqrt[t]).to(device)

    eps_theta = model(x, t)
    #得到均值
    mean = (1 / (1 - betas[t]).sqrt().to(device)) * (x - (coeff * eps_theta))

    z = torch.randn_like(x).to(device)
    sigma_t = betas[t].sqrt().to(device)
    #得到sample的分布
    sample = mean + sigma_t * z

    return (sample)

#从xt恢复x0
def p_sample_loop(model, shape, n_steps, betas, one_minus_alphas_bar_sqrt,device):
    """从x[T]恢复x[T-1]、x[T-2]|...x[0]"""
    cur_x = torch.randn(shape).to(device)
    x_seq = [cur_x]
    # 返回一个反转的迭代器
    for i in reversed(range(n_steps)):
        cur_x = p_sample(model, cur_x, i, betas, one_minus_alphas_bar_sqrt,device)
        x_seq.append(cur_x)
    return x_seq

In [ ]:
# difussion损失函数
def diffusion_loss_fn(model, x_0, alphas_bar_sqrt, one_minus_alphas_bar_sqrt, n_steps):
    """对任意时刻t进行采样计算loss"""
    device=x_0.device
    batch_size = x_0.shape[0]

    # 对一个batchsize样本生成随机的时刻t，t变得随机分散一些，一个batch size里面覆盖更多的t
    t = torch.randint(0, n_steps, size=(batch_size // 2,))
    t = torch.cat([t, n_steps - 1 - t], dim=0)# t的形状（bz）
    t = t.unsqueeze(-1).unsqueeze(-1)# t的形状（batch_size,1）
    t_minus_1 = t - 1

    # 生成随机噪音eps
    e = torch.randn_like(x_0).to(device)

    # x0的系数，根号下(alpha_bar_t)
    a = alphas_bar_sqrt[t].to(device)
    # eps的系数,根号下(1-alpha_bar_t)
    aml = one_minus_alphas_bar_sqrt[t].to(device)
    # 构造模型的输入
    #得到xt时刻的加噪图像

    x = x_0 * a + e * aml

    #当t-1=-1时，x_0的系数为1，eps的系数为0，当t-1!=0时，x_0的系数为根号下(alpha_bar_t-1)，eps的系数为根号下(1-alpha_bar_t-1)
    a_1 = alphas_bar_sqrt[t_minus_1].to(device)
    aml_1 = one_minus_alphas_bar_sqrt[t_minus_1].to(device)
    #得到xt-1时刻的加噪图像
    x_t_minus_1 = x_0 * a_1 + e * aml_1


    #t=t.squeeze(-1).squeeze(-1)
    # 送入模型，得到t时刻的随机噪声预测值
    output = model(x, t.squeeze(-1).squeeze(-1).to(device))

    # 与真实噪声一起计算误差，求平均值
    #返回损失，模型输出的噪声，t时刻加噪的x，t-1时刻加噪的x，以及时间t
    return F.mse_loss(e,output),output,x,x_t_minus_1,t

In [ ]:
norm=False
#选择填充方式pad_0或者interp
pad_pattern='pad_0'

if norm:
    data = pd.read_csv('./data/'+pad_pattern+'/generate_norm.csv')
else:
    data = pd.read_csv('./data/'+pad_pattern+'/generate.csv')

array = np.array(data)
# 创建StandardScaler对象并拟合数据
scaler = StandardScaler()
scaler.fit(array)
# 对数据进行标准化
array = scaler.transform(array)
dataset = torch.tensor(array, dtype=torch.float64).float()
dataset=dataset.unsqueeze(1)
print(dataset.shape)
#(batch_size,1,序列长度)

batch_size = 128
# dataset放到dataloader中
dataloader = torch.utils.data.DataLoader(dataset, batch_size=batch_size, shuffle=True,drop_last=True)

In [ ]:
print('Training model...')
# 迭代周期
num_epoch = 2000
#实例化模型，传入一个数
model = Unet1D(dim=32,channels=1).to(device)  
model_D = Discriminator(dim=32,channels=1).to(device)  
optimizer = torch.optim.Adam(model.parameters(), lr=0.0002)
optimizerD = torch.optim.Adam(model_D.parameters(), lr=0.0002) 
loss_function=nn.BCELoss().to(device)  
# epoch遍历


In [ ]:
dif_loss=[]
d_loss=[]
g_loss=[]
for i in range(num_epoch):
    # dataloader遍历
    epoch_dif_loss=0
    epoch_d_loss=0
    epoch_g_loss=0
    for idx, batch_x in enumerate(dataloader):
        #batch_x （128，2）
        #得到loss,noise和output
        batch_x=batch_x.to(device)
        loss,output,x,x_t_minus_1,t = diffusion_loss_fn(model, batch_x, alphas_bar_sqrt, one_minus_alphas_bar_sqrt, num_steps)
        epoch_dif_loss+=loss.item()

        #根据预测的噪声去噪
        coeff = (betas[t] / one_minus_alphas_bar_sqrt[t]).to(device)
        #得到均值
        mean = (1 / (1 - betas[t]).sqrt().to(device)) * (x - (coeff * output))
        z = torch.randn_like(x).to(device)
        sigma_t = betas[t].sqrt().to(device)
        #得到去噪的图像
        sample = mean + sigma_t * z

        t=t.squeeze(-1).squeeze(-1).to(device)
        # 优化判别器
        optimizerD.zero_grad()
        #使用真实数据训练判别器
        output_real_noise = model_D(x_t_minus_1.to(device),t)
        loss_real=loss_function(output_real_noise,torch.ones_like(output_real_noise).to(device))
        loss_real.backward()
        #使用output训练判别器
        output_fake_noise = model_D(sample.detach().to(device),t)
        loss_fake=loss_function(output_fake_noise,torch.zeros_like(output_fake_noise).to(device))
        loss_fake.backward()
        epoch_d_loss+=(loss_fake.item()+loss_real.item())
        optimizerD.step()

        # 优化生成器difussion
        optimizer.zero_grad()
        output_fake_noise = model_D(sample.to(device),t)
        loss_fake=loss_function(output_fake_noise,torch.ones_like(output_fake_noise).to(device))
        epoch_g_loss+=loss_fake.item()
        loss.backward(retain_graph=True)
        loss_fake.backward()
        optimizer.step()

    epoch_dif_loss=epoch_dif_loss/len(dataloader)
    epoch_d_loss=epoch_d_loss/len(dataloader)
    epoch_g_loss=epoch_g_loss/len(dataloader)
    dif_loss.append(epoch_dif_loss)
    d_loss.append(epoch_d_loss)
    g_loss.append(epoch_g_loss)
    #每100步打印效果
    if (i % 20 == 0):
        print(i,loss)

In [ ]:
plt.plot(dif_loss)

In [ ]:
plt.plot(d_loss)

In [ ]:
plt.plot(g_loss)

In [ ]:
if norm==False:
    torch.save(model.state_dict(),'./'+pad_pattern+'/difussion_gan-difussion{}.pth'.format(num_steps))
    torch.save(model_D.state_dict(),'./'+pad_pattern+'/difussion_gan-discriminator{}.pth'.format(num_steps))
else:
    torch.save(model.state_dict(),'./'+pad_pattern+'/difussion_gan-difussion_norm{}.pth'.format(num_steps))
    torch.save(model_D.state_dict(),'./'+pad_pattern+'/difussion_gan-discriminator_norm{}.pth'.format(num_steps))

In [ ]:
if norm==False:
    model.load_state_dict(torch.load('./'+pad_pattern+'/difussion_gan-difussion{}.pth'.format(num_steps)))
else:
    model.load_state_dict(torch.load('./'+pad_pattern+'/difussion_gan-difussion_norm{}.pth'.format(num_steps)))

In [ ]:


#统计采样时间
start=time.time()
with torch.no_grad():
    x_seq = p_sample_loop(model, [500,1,56], num_steps, betas, one_minus_alphas_bar_sqrt,'cpu')
    x_seq[-1].shape
end=time.time()
print(end-start)

In [ ]:
x_seq[-1].shape

In [ ]:
for i in range(0,51,10):
    print(i)
    x=x_seq[i].detach().squeeze(1)
    x=scaler.inverse_transform(x)
    x1=torch.tensor(x,dtype=torch.float64).float()


    plt.plot(x1[0])
    plt.axis('off')
    plt.savefig('C:/Users/45070/Desktop/图片/r_{}.png'.format(i), bbox_inches='tight')
    plt.show()
    
    print(x_seq[i].shape)

In [ ]:

x_seq=x_seq[-1].detach().squeeze(1)
x=scaler.inverse_transform(x_seq)
np.shape(x)
x1=torch.tensor(x,dtype=torch.float64).float()

plt.plot(x[:].transpose())

In [ ]:
n=11
plt.plot(x1[n])
print(x1[n])

In [ ]:
from evalute import mmd

if norm==False:
    y = pd.read_csv('./data/'+pad_pattern+'/generate.csv')
else:
    y = pd.read_csv('./data/'+pad_pattern+'/generate_norm.csv')
y = np.array(y)
y=y[:500]
y=torch.tensor(y,dtype=torch.float64).float()
mmd(x1,y)

In [ ]:
from scipy.spatial.distance import euclidean

from fastdtw import fastdtw

distance, path = fastdtw(x1, y, dist=euclidean)
distance/500

In [ ]:
import scipy
swd=0
for i in range(500):
    swd+=scipy.stats.wasserstein_distance(x1[i], y[i])
swd/500

In [ ]:
from dtw import dtw
l2_norm = lambda x, y: (x - y) ** 2
manhattan_distance = lambda x, y: np.abs(x - y)

dist, cost_matrix, acc_cost_matrix, path = dtw(x1[8], y[0], dist=l2_norm)
dist

In [ ]:
with torch.no_grad():
    x_seq = p_sample_loop(model, [20000,1,56], num_steps, betas, one_minus_alphas_bar_sqrt,'cpu')
x_seq=x_seq[-1].detach()

In [ ]:
x_seq=x_seq.squeeze(1)
x=scaler.inverse_transform(x_seq)


In [ ]:
if norm==False:
    x=np.concatenate((x,np.ones((20000,1))),axis=1)
else:
    x=np.concatenate((x,np.zeros((20000,1))),axis=1)

In [ ]:
dataframe = pd.DataFrame(x)
dataframe

In [ ]:
if norm==False:
    dataframe.to_csv('./data/'+pad_pattern+'/diffusion.csv',index=False)
else:
    dataframe.to_csv('./data/'+pad_pattern+'/diffusion_norm.csv',index=False)